In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/6월/나무문화_6월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

377


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_6월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,7월 개봉 예정 영화 신작 영화추천 신작 영화 추천 극장 개봉 예정 영화 7월 리부...,0,"['7월', '개봉', '예정', '영화', '신작', '영화', '추천', '신작...","[('7월', 'Number'), ('개봉', 'Noun'), ('예정', 'Nou..."
1,문제는 쥬라기 월드 탑건 신비한 동물사전 레전드 영화의 리부트가 연일 관객들의 기대...,0,"['문제', '는', '쥬라기', '월드', '탑건', '신비한', '동물', '사...","[('문제', 'Noun'), ('는', 'Josa'), ('쥬라기', 'Noun'..."
2,어린 시절의 향수를 진하게 불러주는 오마주는 반갑지만 그때 느꼈던 감동과 재미는 다...,0,"['어린', '시절', '의', '향수', '를', '진하게', '불러주는', '오...","[('어리다', 'Verb'), ('시절', 'Noun'), ('의', 'Josa'..."
3,저야 극장에서 보는 어지간한 영화는 재밌게 보고 나오는 편입니다만 그래서 극장 관람...,0,"['저', '야', '극장', '에서', '보는', '어지간한', '영화', '는'...","[('저', 'Noun'), ('야', 'Josa'), ('극장', 'Noun'),..."
4,그래서 2차 3차 가공해서 영화를 해석하고 추천하는 글을 쓰는 데는 생각이 참 많아...,0,"['그래서', '2', '차', '3', '차', '가공', '해서', '영화', ...","[('그래서', 'Adverb'), ('2', 'Number'), ('차', 'No..."
...,...,...,...,...
548086,이런 의문들은 또다시 석기를 그 지역의 환경적 맥락에서 떼어 놓고 보기 때문에 생겨...,11966,"['이런', '의문', '들', '은', '또다시', '석기', '를', '그', ...","[('이렇다', 'Adjective'), ('의문', 'Noun'), ('들', '..."
548087,어떤 종이든 간에 대나무가 사방에 깔린 열대우림 지역에서는 대나무를 이용하는 것이 ...,11966,"['어떤', '종이', '든', '간', '에', '대나무', '가', '사방', ...","[('어떻다', 'Adjective'), ('종이', 'Noun'), ('든', '..."
548088,그렇다면 왜 3만 년 전부터 도구가 변하기 시작하는 것일까 만약 도구를 만든 사람 ...,11966,"['그렇다면', '왜', '3만', '년', '전', '부터', '도구', '가',...","[('그렇다', 'Adjective'), ('왜', 'Noun'), ('3만', '..."
548089,바로 이때부터 진짜로 정교한 현대 호모 사피엔스의 도구가 발명되기 시작한다,11966,"['바로', '이', '때', '부터', '진짜', '로', '정교한', '현대',...","[('바로', 'Noun'), ('이', 'Determiner'), ('때', 'N..."


In [6]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,7월 개봉 예정 영화 신작 영화추천 신작 영화 추천 극장 개봉 예정 영화 7월 리부...,0,"['7월', '개봉', '예정', '영화', '신작', '영화', '추천', '신작...","[('7월', 'Number'), ('개봉', 'Noun'), ('예정', 'Nou...",7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
1,문제는 쥬라기 월드 탑건 신비한 동물사전 레전드 영화의 리부트가 연일 관객들의 기대...,0,"['문제', '는', '쥬라기', '월드', '탑건', '신비한', '동물', '사...","[('문제', 'Noun'), ('는', 'Josa'), ('쥬라기', 'Noun'...",문제 는 쥬라기 월드 탑건 신비한 동물 사전 레전드 영화 의 리부트 가 연일 관객 ...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
2,어린 시절의 향수를 진하게 불러주는 오마주는 반갑지만 그때 느꼈던 감동과 재미는 다...,0,"['어린', '시절', '의', '향수', '를', '진하게', '불러주는', '오...","[('어리다', 'Verb'), ('시절', 'Noun'), ('의', 'Josa'...",어린 시절 의 향수 를 진하게 불러주는 오마주 는 반갑지만 그때 느꼈던 감동 과 재...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
3,저야 극장에서 보는 어지간한 영화는 재밌게 보고 나오는 편입니다만 그래서 극장 관람...,0,"['저', '야', '극장', '에서', '보는', '어지간한', '영화', '는'...","[('저', 'Noun'), ('야', 'Josa'), ('극장', 'Noun'),...",저 야 극장 에서 보는 어지간한 영화 는 재밌게 보고 나오는 편입 니 다만 그래서 ...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
4,그래서 2차 3차 가공해서 영화를 해석하고 추천하는 글을 쓰는 데는 생각이 참 많아...,0,"['그래서', '2', '차', '3', '차', '가공', '해서', '영화', ...","[('그래서', 'Adverb'), ('2', 'Number'), ('차', 'No...",그래서 2 차 3 차 가공 해서 영화 를 해석 하고 추천 하는 글 을 쓰는 데 는 ...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
...,...,...,...,...,...,...
548086,이런 의문들은 또다시 석기를 그 지역의 환경적 맥락에서 떼어 놓고 보기 때문에 생겨...,11966,"['이런', '의문', '들', '은', '또다시', '석기', '를', '그', ...","[('이렇다', 'Adjective'), ('의문', 'Noun'), ('들', '...",이런 의문 들 은 또다시 석기 를 그 지역 의 환경 적 맥락 에서 떼어 놓고 보기 ...,blog 인류 의 위대한 여행 55 개 의 글 인류 의 위대한 여행 목록 닫기 인류...
548087,어떤 종이든 간에 대나무가 사방에 깔린 열대우림 지역에서는 대나무를 이용하는 것이 ...,11966,"['어떤', '종이', '든', '간', '에', '대나무', '가', '사방', ...","[('어떻다', 'Adjective'), ('종이', 'Noun'), ('든', '...",어떤 종이 든 간 에 대나무 가 사방 에 깔린 열대우림 지역 에서는 대나무 를 이용...,blog 인류 의 위대한 여행 55 개 의 글 인류 의 위대한 여행 목록 닫기 인류...
548088,그렇다면 왜 3만 년 전부터 도구가 변하기 시작하는 것일까 만약 도구를 만든 사람 ...,11966,"['그렇다면', '왜', '3만', '년', '전', '부터', '도구', '가',...","[('그렇다', 'Adjective'), ('왜', 'Noun'), ('3만', '...",그렇다면 왜 3만 년 전 부터 도구 가 변하기 시작 하는 것 일까 만약 도구 를 만...,blog 인류 의 위대한 여행 55 개 의 글 인류 의 위대한 여행 목록 닫기 인류...
548089,바로 이때부터 진짜로 정교한 현대 호모 사피엔스의 도구가 발명되기 시작한다,11966,"['바로', '이', '때', '부터', '진짜', '로', '정교한', '현대',...","[('바로', 'Noun'), ('이', 'Determiner'), ('때', 'N...",바로 이 때 부터 진짜 로 정교한 현대 호모 사피 엔 스 의 도구 가 발명 되기 시...,blog 인류 의 위대한 여행 55 개 의 글 인류 의 위대한 여행 목록 닫기 인류...


In [7]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,7월 개봉 예정 영화 신작 영화추천 신작 영화 추천 극장 개봉 예정 영화 7월 리부...,0,"['7월', '개봉', '예정', '영화', '신작', '영화', '추천', '신작...","[('7월', 'Number'), ('개봉', 'Noun'), ('예정', 'Nou...",7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
1,문제는 쥬라기 월드 탑건 신비한 동물사전 레전드 영화의 리부트가 연일 관객들의 기대...,0,"['문제', '는', '쥬라기', '월드', '탑건', '신비한', '동물', '사...","[('문제', 'Noun'), ('는', 'Josa'), ('쥬라기', 'Noun'...",문제 는 쥬라기 월드 탑건 신비한 동물 사전 레전드 영화 의 리부트 가 연일 관객 ...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
2,어린 시절의 향수를 진하게 불러주는 오마주는 반갑지만 그때 느꼈던 감동과 재미는 다...,0,"['어린', '시절', '의', '향수', '를', '진하게', '불러주는', '오...","[('어리다', 'Verb'), ('시절', 'Noun'), ('의', 'Josa'...",어린 시절 의 향수 를 진하게 불러주는 오마주 는 반갑지만 그때 느꼈던 감동 과 재...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
3,저야 극장에서 보는 어지간한 영화는 재밌게 보고 나오는 편입니다만 그래서 극장 관람...,0,"['저', '야', '극장', '에서', '보는', '어지간한', '영화', '는'...","[('저', 'Noun'), ('야', 'Josa'), ('극장', 'Noun'),...",저 야 극장 에서 보는 어지간한 영화 는 재밌게 보고 나오는 편입 니 다만 그래서 ...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
4,그래서 2차 3차 가공해서 영화를 해석하고 추천하는 글을 쓰는 데는 생각이 참 많아...,0,"['그래서', '2', '차', '3', '차', '가공', '해서', '영화', ...","[('그래서', 'Adverb'), ('2', 'Number'), ('차', 'No...",그래서 2 차 3 차 가공 해서 영화 를 해석 하고 추천 하는 글 을 쓰는 데 는 ...,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
...,...,...,...,...,...,...
548086,이런 의문들은 또다시 석기를 그 지역의 환경적 맥락에서 떼어 놓고 보기 때문에 생겨...,11966,"['이런', '의문', '들', '은', '또다시', '석기', '를', '그', ...","[('이렇다', 'Adjective'), ('의문', 'Noun'), ('들', '...",이런 의문 들 은 또다시 석기 를 그 지역 의 환경 적 맥락 에서 떼어 놓고 보기 ...,blog 인류 의 위대한 여행 55 개 의 글 인류 의 위대한 여행 목록 닫기 인류...
548087,어떤 종이든 간에 대나무가 사방에 깔린 열대우림 지역에서는 대나무를 이용하는 것이 ...,11966,"['어떤', '종이', '든', '간', '에', '대나무', '가', '사방', ...","[('어떻다', 'Adjective'), ('종이', 'Noun'), ('든', '...",어떤 종이 든 간 에 대나무 가 사방 에 깔린 열대우림 지역 에서는 대나무 를 이용...,blog 인류 의 위대한 여행 55 개 의 글 인류 의 위대한 여행 목록 닫기 인류...
548088,그렇다면 왜 3만 년 전부터 도구가 변하기 시작하는 것일까 만약 도구를 만든 사람 ...,11966,"['그렇다면', '왜', '3만', '년', '전', '부터', '도구', '가',...","[('그렇다', 'Adjective'), ('왜', 'Noun'), ('3만', '...",그렇다면 왜 3만 년 전 부터 도구 가 변하기 시작 하는 것 일까 만약 도구 를 만...,blog 인류 의 위대한 여행 55 개 의 글 인류 의 위대한 여행 목록 닫기 인류...
548089,바로 이때부터 진짜로 정교한 현대 호모 사피엔스의 도구가 발명되기 시작한다,11966,"['바로', '이', '때', '부터', '진짜', '로', '정교한', '현대',...","[('바로', 'Noun'), ('이', 'Determiner'), ('때', 'N...",바로 이 때 부터 진짜 로 정교한 현대 호모 사피 엔 스 의 도구 가 발명 되기 시...,blog 인류 의 위대한 여행 55 개 의 글 인류 의 위대한 여행 목록 닫기 인류...


In [8]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,7월 개봉 예정 영화 신작 영화 추천 신작 영화 추천 극장 개봉 예정 영화 7월 리...
1,1,6월 20일 지인 이 파니 를 위해 카메라 를 닦고 닦고 또 닦아 완전 새거처럼 만...
2,2,2 박 3일 파주 여행 을 하고 온 담담 이번 여행 은 혼자 떠나기도 했고 요즘 바...
3,3,안녕하세요 연 쓰입니다 오늘 은 좀 특별한 이색 카페 를 다녀왔어요 연 쓰는 평소 ...
4,4,9 30 18 00 주차 가능 안녕하세요 챠넷입니 다 D 장마 가 시작 되면서 비 ...
...,...,...
11958,11962,신영복 평전 1 책 소개 75년 신영복 의 삶 을 정리 하다 돌아간 지 4년 이 흘...
11959,11963,안녕하세요 달꽃 틔움 숲 이에요 현재 달꽃 틔움 숲 에서는 다양한 커리큘럼 의 클래...
11960,11964,춘천 여행 레고랜드 남이섬 말고 여기어 때 강원도 춘천시 신동면 김유정 로 1435...
11961,11965,오늘 의 답사 코스 열세 번째 유네스코 세계 유산 한국 의 산사 영주 부석사 를 답...


In [9]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [10]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [11]:
tdm.shape

(11963, 337984)

In [13]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04692719, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.02963695, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04974233, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0060287 , 0.00563125, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [14]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,337974,337975,337976,337977,337978,337979,337980,337981,337982,337983
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.046927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.070597,0.115401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11958,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11959,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11960,0.000000,0.029637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11961,0.000000,0.049742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [16]:
tfidf_DF

,0,00,000,0001,0001076147,0001533770,0001538418,00016,0001731766,0001734189,...,힝,힝구,힝링구,힝뿌비,힝스,힝쟁,힠,힡,힣,힣힣
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.046927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.070597,0.115401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11958,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11959,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11960,0.000000,0.029637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11961,0.000000,0.049742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/6월/나무문화_6월_TF-IDF행렬(원본).txt", index=False, encoding='UTF-8')

In [ ]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

In [49]:
# 한글로만 이루어진 TF-IDF 행렬도 txt로 저장해줌.
korean_df.to_csv("../dataset/2022/본문_TF-IDF/6월/나무문화_6월_TF-IDF행렬(원본).txt", index=False, encoding='UTF-8')

In [50]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,51.032181
1,00,50.503403
2,000,19.454386
3,0001,0.049739
4,0001076717,0.034615
...,...,...
134284,힙니,0.062174
134285,힙스터,0.058074
134286,힙합,0.405733
134287,힝,0.086247


In [51]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/6월/나무문화_6월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')